In [ ]:
import pandas as pd
import random
from faker import Faker
import numpy as np
from datetime import timedelta

# Initialize Faker
fake = Faker()

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

def generate_ids(num, prefix='ID'):
    return [f"{prefix}{str(i).zfill(4)}" for i in range(1, num + 1)]

# Configuration for number of records
NUM_DEPARTMENTS = 10
NUM_EMPLOYEES = 50
NUM_CLIENTS = 40
NUM_PROBLEMS = 60
NUM_SERVICES = 30
NUM_THREATS = 25

# 1. Department Table
def create_departments():
    return pd.DataFrame({
        'dept_id': generate_ids(NUM_DEPARTMENTS, 'DEPT'),
        'dept_head': [fake.name() for _ in range(NUM_DEPARTMENTS)]
    })

# 2. Employee Table
def create_employees():
    emp_base = pd.DataFrame({
        'emp_id': generate_ids(NUM_EMPLOYEES, 'EMP'),
        'ssn': [fake.unique.random_number(digits=9) for _ in range(NUM_EMPLOYEES)],
        'name': [fake.name() for _ in range(NUM_EMPLOYEES)],
        'address': [fake.address().replace('\n', ', ') for _ in range(NUM_EMPLOYEES)]
    })
    
    # Create separate employee-department assignment table
    emp_dept = pd.DataFrame({
        'emp_id': emp_base['emp_id'],
        'dept_id': random.choices(generate_ids(NUM_DEPARTMENTS, 'DEPT'), k=NUM_EMPLOYEES),
        'start_date': [fake.date_between(start_date='-3y', end_date='today') for _ in range(NUM_EMPLOYEES)]
    })
    
    # Create separate employee-manager relationship table
    managers = emp_base['emp_id'].tolist()[:5]  # First 5 employees are managers
    emp_manager = pd.DataFrame({
        'emp_id': emp_base['emp_id'][5:],  # Excluding managers
        'manager_id': [random.choice(managers) for _ in range(NUM_EMPLOYEES-5)],
        'assignment_date': [fake.date_between(start_date='-2y', end_date='today') 
                          for _ in range(NUM_EMPLOYEES-5)]
    })
    
    return emp_base, emp_dept, emp_manager

# 3. Client Table
def create_clients():
    client_base = pd.DataFrame({
        'client_id': generate_ids(NUM_CLIENTS, 'CLT'),
        'name': [fake.company() for _ in range(NUM_CLIENTS)],
        'address': [fake.address().replace('\n', ', ') for _ in range(NUM_CLIENTS)]
    })
    
    # Create separate client-employee assignment table
    client_emp = pd.DataFrame({
        'client_id': random.choices(client_base['client_id'], k=NUM_CLIENTS//2),
        'emp_id': [f"EMP{str(random.randint(1, NUM_EMPLOYEES)).zfill(4)}" 
                  for _ in range(NUM_CLIENTS//2)],
        'assignment_date': [fake.date_between(start_date='-1y', end_date='today') 
                          for _ in range(NUM_CLIENTS//2)]
    })
    
    return client_base, client_emp

# 4. Problems Table
def create_problems():
    return pd.DataFrame({
        'problem_id': generate_ids(NUM_PROBLEMS, 'PRB'),
        'threat_id': random.choices(generate_ids(NUM_THREATS, 'THR'), k=NUM_PROBLEMS)
    })

# 5. Services Table
def create_services():
    return pd.DataFrame({
        'service_id': generate_ids(NUM_SERVICES, 'SVC'),
        'dept_id': random.choices(generate_ids(NUM_DEPARTMENTS, 'DEPT'), k=NUM_SERVICES),
        'problem_id': random.choices(generate_ids(NUM_PROBLEMS, 'PRB'), k=NUM_SERVICES)
    })

# 6. Solves Table
def create_solves():
    return pd.DataFrame({
        'problem_id': random.choices(generate_ids(NUM_PROBLEMS, 'PRB'), k=NUM_PROBLEMS),
        'service_id': random.choices(generate_ids(NUM_SERVICES, 'SVC'), k=NUM_PROBLEMS),
        'date_assigned': [fake.date_between(start_date='-1y', end_date='today') for _ in range(NUM_PROBLEMS)]
    }).drop_duplicates()

# 7. Faces Table
def create_faces():
    return pd.DataFrame({
        'client_id': random.choices(generate_ids(NUM_CLIENTS, 'CLT'), k=NUM_PROBLEMS),
        'problem_id': generate_ids(NUM_PROBLEMS, 'PRB'),
        'status': [random.choice(['OPEN', 'IN_PROGRESS', 'PENDING', 'RESOLVED', 'CLOSED']) for _ in range(NUM_PROBLEMS)]
    })

# 8. Handles Table
def create_handles():
    return pd.DataFrame({
        'dept_id': random.choices(generate_ids(NUM_DEPARTMENTS, 'DEPT'), k=NUM_PROBLEMS),
        'problem_id': generate_ids(NUM_PROBLEMS, 'PRB'),
        'level': [random.choice(['LOW', 'MEDIUM', 'HIGH']) for _ in range(NUM_PROBLEMS)]
    })

# 9. Threat_Identification Table
def create_threat_identification():
    return pd.DataFrame({
        'threat_id': generate_ids(NUM_THREATS, 'THR'),
        'type': [random.choice(['Malware', 'Phishing', 'DDoS', 'SQL Injection', 'Zero-day',
                   'Ransomware', 'Social Engineering', 'Man-in-the-middle']) for _ in range(NUM_THREATS)],
        'security_level': [random.choice(['LOW', 'MEDIUM', 'HIGH']) for _ in range(NUM_THREATS)],
        'problem_id': random.choices(generate_ids(NUM_PROBLEMS, 'PRB'), k=NUM_THREATS),
        'service_id': random.choices(generate_ids(NUM_SERVICES, 'SVC'), k=NUM_THREATS)
    })

# 10. Security_Measures Table
def create_security_measures():
    return pd.DataFrame({
        'service_id': generate_ids(NUM_SERVICES, 'SVC'),
        'sec_type': [random.choice(['Firewall', 'Encryption', 'Authentication', 'Access Control',
                    'Monitoring', 'Backup', 'Patch Management', 'Training']) for _ in range(NUM_SERVICES)],
        'cost': [round(random.uniform(1000, 10000), 2) for _ in range(NUM_SERVICES)],
        'complexity': [random.choice(['LOW', 'MEDIUM', 'HIGH']) for _ in range(NUM_SERVICES)]
    })

# 11. Immediate_Response_Team Table
def create_immediate_response_team():
    return pd.DataFrame({
        'dept_id': generate_ids(NUM_DEPARTMENTS, 'DEPT'),
        'response_time': [random.randint(5, 60) for _ in range(NUM_DEPARTMENTS)]  # Response time in minutes
    })

# 12. Skill_Level Table
def create_skill_level():
    return pd.DataFrame({
        'dept_id': generate_ids(NUM_DEPARTMENTS, 'DEPT'),
        'skill_level': [random.choice(['Beginner', 'Intermediate', 'Advanced', 'Expert']) for _ in range(NUM_DEPARTMENTS)]
    })

# 13. Threat_Analysis Table
def create_threat_analysis():
    return pd.DataFrame({
        'dept_id': generate_ids(NUM_DEPARTMENTS, 'DEPT'),
        'methodology': [random.choice(['Qualitative', 'Quantitative', 'Hybrid']) for _ in range(NUM_DEPARTMENTS)],
        'technology': [random.choice(['AI', 'Machine Learning', 'Behavioral Analysis', 'Heuristics']) for _ in range(NUM_DEPARTMENTS)]
    })

# 14. Threat_Solutions Table
def create_threat_solutions():
    return pd.DataFrame({
        'dept_id': generate_ids(NUM_DEPARTMENTS, 'DEPT'),
        'solution_type': [random.choice(['Patch Management', 'Threat Intelligence', 'Vulnerability Assessment']) for _ in range(NUM_DEPARTMENTS)]
    })

# Generate all tables
def generate_normalized_database():
    # Generate all tables
    departments = create_departments()
    emp_base, emp_dept, emp_manager = create_employees()
    client_base, client_emp = create_clients()
    problems = create_problems()
    services = create_services()
    solves = create_solves()
    faces = create_faces()
    handles = create_handles()
    threat_identification = create_threat_identification()
    security_measures = create_security_measures()
    immediate_response_team = create_immediate_response_team()
    skill_level = create_skill_level()
    threat_analysis = create_threat_analysis()
    threat_solutions = create_threat_solutions()
    
    # Create dictionary of all tables
    tables = {
        'departments': departments,
        'employees': emp_base,
        'employee_departments': emp_dept,
        'employee_managers': emp_manager,
        'clients': client_base,
        'client_employees': client_emp,
        'problems': problems,
        'services': services,
        'solves': solves,
        'faces': faces,
        'handles': handles,
        'threat_identification': threat_identification,
        'security_measures': security_measures,
        'immediate_response_team': immediate_response_team,
        'skill_level': skill_level,
        'threat_analysis': threat_analysis,
        'threat_solutions': threat_solutions
    }
    
    return tables

# Generate and save the data
if __name__ == "__main__":
    # Create output directories
    import os
    os.makedirs('normalized_csv1', exist_ok=True)
    os.makedirs('normalized_excel1', exist_ok=True)
    
    # Generate tables
    tables = generate_normalized_database()
    
    # Save to CSV files
    for name, df in tables.items():
        df.to_csv(f'normalized_csv1/{name}.csv', index=False)
        print(f"Saved {name}.csv")
    
    # Save to Excel (single file with multiple sheets)
    with pd.ExcelWriter('normalized_excel1/security_database_normalized1.xlsx') as writer:
        for name, df in tables.items():
            df.to_excel(writer, sheet_name=name, index=False)
        print("Saved combined Excel file")


Saved departments.csv
Saved employees.csv
Saved employee_departments.csv
Saved employee_managers.csv
Saved clients.csv
Saved client_employees.csv
Saved problems.csv
Saved services.csv
Saved solves.csv
Saved faces.csv
Saved handles.csv
Saved threat_identification.csv
Saved security_measures.csv
Saved immediate_response_team.csv
Saved skill_level.csv
Saved threat_analysis.csv
Saved threat_solutions.csv
Saved combined Excel file
